## Решите следующие задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):

1.	Найти велосипед с максимальным временем пробега.
2.	Найти наибольшее геодезическое расстояние между станциями.
3.	Найти путь велосипеда с максимальным временем пробега через станции.
4.	Найти количество велосипедов в системе.
5.	Найти пользователей потративших на поездки более 3 часов.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark
from pyspark.sql import functions as F

In [11]:
spark = SparkSession.builder.appName("lr2").config("spark.sql.legacy.timeParserPolicy", "LEGACY").getOrCreate()

In [12]:
trips_df = spark.read.option("header", "true").option("inferSchema", "true").csv("trip.csv")
print(trips_df.dtypes)

stations_df = spark.read.option("header", "true").option("inferSchema", "true").csv("station.csv")
print(stations_df.dtypes)

[('id', 'int'), ('duration', 'int'), ('start_date', 'string'), ('start_station_name', 'string'), ('start_station_id', 'int'), ('end_date', 'string'), ('end_station_name', 'string'), ('end_station_id', 'int'), ('bike_id', 'int'), ('subscription_type', 'string'), ('zip_code', 'string')]
[('id', 'int'), ('name', 'string'), ('lat', 'double'), ('long', 'double'), ('dock_count', 'int'), ('city', 'string'), ('installation_date', 'string')]


In [28]:
result = (trips_df
          .groupBy("bike_id")
          .agg((F.sum("duration")/3600).alias("tot_duration"))
          .orderBy(F.desc("tot_duration"))
          .limit(1)).collect()[0]

print(result)

Row(bike_id=535, tot_duration=5169.914722222222)


In [33]:
from pyspark.sql.functions import max, radians, sin, cos, acos, col

def calc_dist(lat1, lon1, lat2, lon2):
    R = 6371 # Радиус Земли
    lat1, lat2, lon1, lon2 = radians(lat1), radians(lat2), radians(lon1), radians(lon2)
    return R * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon2 - lon1))

max_dist_stations = (stations_df.alias("st1")
            .crossJoin(stations_df.alias("st2"))
            .filter(col("st1.id")<col("st2.id"))
            .withColumn("dist",calc_dist(col("st1.lat"), col("st1.long"), col("st2.lat"),col("st2.long")))
            .orderBy(F.desc("dist"))
            .limit(1)).collect()[0]["dist"]

print(max_dist_stations)

69.92087595421542


In [34]:
result = (trips_df
          .groupBy("bike_id")
          .agg((F.sum("duration")/3600).alias("tot_duration"))
          .orderBy(F.desc("tot_duration"))
          .limit(1))

max_bike_duration = result.collect()
bike_id_max = max_bike_duration[0]["bike_id"]
total_duration = max_bike_duration[0]["tot_duration"]

bike_trips = (trips_df.filter(F.col("bike_id")==bike_id_max)
              .select("start_date", "start_station_name", "end_date", "end_station_name")
              .orderBy("start_date"))
bike_trips.show()


+---------------+--------------------+---------------+--------------------+
|     start_date|  start_station_name|       end_date|    end_station_name|
+---------------+--------------------+---------------+--------------------+
| 1/1/2014 13:42|Mechanics Plaza (...| 1/1/2014 14:36|Embarcadero at Sa...|
| 1/1/2014 18:51|Embarcadero at Sa...| 1/1/2014 19:13|       Market at 4th|
| 1/1/2014 19:48|       Market at 4th| 1/1/2014 20:01|South Van Ness at...|
|1/10/2014 20:13|      Market at 10th|1/10/2014 20:17|  Powell Street BART|
| 1/10/2014 8:09|Embarcadero at Fo...| 1/10/2014 8:19|San Francisco Cal...|
| 1/10/2014 8:21|San Francisco Cal...| 1/10/2014 8:31|Temporary Transba...|
| 1/10/2014 9:19|Temporary Transba...| 1/10/2014 9:32|      Market at 10th|
|1/11/2014 19:06|  Powell Street BART|1/11/2014 19:11|      Market at 10th|
|1/12/2014 12:21|      Market at 10th|1/12/2014 12:29|       Market at 4th|
|1/12/2014 17:36|       Market at 4th|1/12/2014 17:58|San Francisco Cal...|
|1/13/2014 1

In [43]:
tot_bikes = (trips_df.select("bike_id").distinct().count())
print(tot_bikes)

700


In [54]:
result = (trips_df
          .groupBy("zip_code")
          .agg((F.sum("duration")/3600).alias("tot_duration"))
          .filter(F.col("zip_code")!="nil")
          .filter(F.col("tot_duration")>3)
          .orderBy(F.desc("tot_duration"))
          .select("zip_code","tot_duration")
        )
result.show()

+--------+------------------+
|zip_code|      tot_duration|
+--------+------------------+
|   94107|13821.433888888889|
|   94105| 7110.035555555555|
|   94133| 6010.465277777777|
|   94102| 5313.339166666667|
|   94103| 5313.163333333333|
|   95531| 4797.333333333333|
|   94111|3956.9436111111113|
|   95112|3539.5472222222224|
|   94109| 3349.202222222222|
|   94040|2168.8683333333333|
|   94110| 2061.648888888889|
|   94117|1917.0313888888888|
|   94301|1830.6605555555554|
|   94041|1743.4122222222222|
|   94158|1735.6019444444444|
|   94306|1541.8452777777777|
|   94025|1438.3991666666666|
|   94108|1424.3227777777777|
|   94611|1393.0294444444444|
|   94010|1333.3772222222221|
+--------+------------------+
only showing top 20 rows

